# Function Part 3
---
We already know how powerful the UDF could be. We could build any kind of UDF based on our requirements. So far, UDF seems perfect, but let's think about that if we are facing a big project, we need to build a lot of UDFs to help us take care of the different problems. For an example, we need to build 50 UDFs and some of them are higher-order functions. In this case, the first problem is how to figure out 50 different meanful and readable functions' names for each function? Another negative consequence of this approach is that for each individual UDF, they could have different arguments. If you want to pass one UDF's result to other one, you may need to figure out the arugument's counts and form. Sometime, when our UDFs are complex this problem could be tough to solve. Fortunally, nested function could tackle them. We will have a deep understanding for this concept by following example.

Everyone knows that the square roots of 4 are 2 and -2. In R, we use `sqrt()` to get the postive square root value for a numeric vector. 

In [1]:
sqrt(4)

[1] 2

We should know that a numeric has two squart root value, one is positive, and another is negative.
\begin{equation}
2^2 = 4 
\end{equation}
\begin{equation}
-2^2 = 4 
\end{equation}

Let's see this square root value deeper, how to compute the square value? Here is one simple alogrithem.
1. Suppose we have a number which is a, and we want to know the get the square root of a.
2. We initialize x equals 1.
3. When ${x}^{2}\neq a$, we use update function to update x's value.
4. The update formula is $\frac{x+\frac{a}{x}}{2}$
5. Return x which is the sauqre root of a.

This alogrithem is naive and has some bugs for some cases, we will fix them later, but now this alogrithem is enough to show us the basic concept.

Let's see the implementation for this alogrithem.

In [2]:
# square root function
square_root <- function(a) {
  x = 1
  while (x * x != a) {
    x = square_root_update(x, a)
  }
  return(x)
}
# update function
square_root_update <- function(x, a){
  return((x + a/x) / 2)
}

In [3]:
square_root(16)

[1] 4

In [5]:
square_root(2.25)

[1] 1.5

To this end, this alogrithem cannot handle a numeric value with an irrational square root value. If we call this function to 2. The alogrithem will keep running and return us an error, because the condition statement in `while` loop cannot be required. Most time, we use 1.414 to represent the squar root value for 2, which means we only need an approxmation to replace the *ture* square root value. So far, we will do some updates for this alogrithem to solve the infinite loop. 

Updating the condition by following formula.
\begin{equation}
while\quad abs(a - x^x) > tolerance
\end{equation}
The tolerance is a very small value, in this case, we set it to ${10}^{-3}$. The samller value you set, the more accurate result you get.

In [5]:
approx_eq <- function(x, y, tolerance = 1e-3) {
  flag = FALSE
  if (x**2 == y) {
    flag
  } else if (abs(x**2 - y) > tolerance){
    flag = TRUE
  }
  return (flag)
}

# Update square_root function
square_root <- function(a) {
  x = 1
  while (approx_eq(x=x, y=a)) {
    x = square_root_update(x, a)
  }
  return(round(x, 3))
}

In [6]:
square_root(2)

[1] 1.414

Let us check more details on these functions, when we call `approx_eq`, we pass `x` and `a` to this function. This x is the return value of function`square_root_update`, and y equals a. We pass one functions's return valus to anothet function, so we need to set arguments correctly. However, the arguments could be complex in some cases. So, how could we avoid this trouble? The answer is nested function.

Nested function is a kind of structure which is we build functions within one funcion. We are introducing two new concepts here to help us have a better insight for nested function.

The **global function** and **local function**. As the name suggests, when a variable is in the global enviroment, we coulld call it golbal function, however, we cannot call a local variable in global enviroment. Here is s simple demo.

In [7]:
global_function <- function() { 
  x <- 10
  print(paste("The outler number is", x))# line 1
  
  local_function <- function(y = 1) { # line 2
    print(paste("The inner number is", y)) # line 3
  }
  return (local_function()) # line 4
}

The `global_function` is a nested function and global function. The `local_function` is a function in local enviroment. Also, we could call it as local function.

When we call a local function in global, R will return us an error.
```r
> local_function(y=10)
Error in local_function(y = 10) :
  could not find function "local_function"
```

```r
> x
Error: object 'x' not found
```

```r
> global_function()
[1] "The outler number is 10"
[1] "The inner number is 1"
```
The logic for `global_function` is by following order.

line1 >>> line2 >>> line4 >>> line2 >>> line3

The main idea is that when we created a nested function, the inner functions are local functions, all variables defined within a function are also local variables. When we call functions and variables, be careful about it.

Let's rewrite our `square_root` function to a nested function.

In [8]:
# Get average value for two numbers
#' @param x A number.
#' @param y A number.
average <- function(x, y) { #line 1
  return((x + y) / 2) #line 2
}
 
# improve function will determin whether update our innitial value no not and return the final result
#' @param update a function
#' @param close a function
#' @param guess a number
improve <- function(update, close, guess = 1) {#line 3
  while(close(guess) == FALSE) {#line 4
      guess = update(guess)#line 5
  }
  return(round(guess, 3)) # this is our square root value    #line 6
}

# close function help us do determin whether the current ans is qualified or not
#' @param x A number.
#' @param y A number.
#' @param tolerance A number.
approx_eq <-function(x, y, tolerance = 1e-3) {#line 7
  return(abs(x - y) < tolerance)#line 8
}

# sqrt_root function
#' @param a A number we want to know its square root value
sqrt_root <- function(a){#line 9
  sqrt_update <- function(x) {#line 10
    return(average(x, a/x))#line 11
  }
  sqrt_close <- function(x) {#line 12
    return(approx_eq(x * x, a))#line 13
  }
  return(improve(sqrt_update, sqrt_close))#line 14
}

OK, we build some funtions, `average`, `improve`,and `approx_eq` are in the global enviroment. The `sqrt_root` is a nested function, it contains two inner function which are in the local envrioment. Even they are in the local enviroment, they still could call other functions which are in the global enviroment.

Let's see the logic of these functions.
1. When we call `sqrt_root`, the line 9.
2. line 10
3. line 12
4. line 14, call `improve` function.
5. line 3
6. line 5, call close function which is `sqrt_close` function
7. line 12
8. line 13, call `approx_eq` function
9. line 7
10. line 8, will return TRUE and FALSE to line 4
11. If line 8's result is FALSE, then line 6, the progrm end and return guess for result.
12. If line 8's result is TRUE then line 5, call `sqrt_update` function, the line 10.
13. line 10
14. line 11, call `average` function, line 1.
15. line 2 return average value of x and $x/a$
16. so far, we have a new guess value from line2.
17. repeat setp 4 to step 16 until step 10 result is FALSE and return answer.

For step 15 and 16, they are the power of nested function. line 2 result directly pass to line 14 the `improve` function. So, the program repeat automatocally and we do not need to worry about the argument in each function.